# 2.A. Cleaning and Engineering -- Data Merging

In [199]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import requests
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import lyricsgenius
import re
import sys
import spotipy
import spotipy.util as util
from sklearn.compose import ColumnTransformer, make_column_transformer
from pyjarowinkler import distance


import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import requests
import seaborn as sns
import warnings
import matplotlib.pyplot as plt

import nltk
from nltk.stem import PorterStemmer
from sklearn.model_selection import cross_val_score

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB


In [274]:
spotify = pd.read_csv('spotify_final_230k.csv')

In [273]:
genius = pd.read_csv('saturday_done.csv')

In [202]:
spotify.shape, genius.shape

((240556, 34), (45573, 7))

In [203]:
genius.isnull().sum()

album         6806
artist           0
date         12703
features         0
lyrics           4
producers        0
song             0
dtype: int64

In [204]:
genius['artist_album_track_key'] = genius['artist'] + '|'  + genius['song']
spotify['artist_album_track_key'] =  spotify['arist_y'] + '|'  + spotify['track_name']

genius['artist_album_track_key'] = genius['artist_album_track_key'].str.replace(' ', '_')
spotify['artist_album_track_key'] = spotify['artist_album_track_key'].str.replace(' ', '_')

In [249]:
genius.columns

Index(['album', 'artist', 'date', 'features', 'lyrics', 'producers', 'song',
       'artist_album_track_key', 'artist_clean', 'album_name_clean',
       'artist_album_clean', 'song_name_clean', 'artist_album_track_clean_key',
       'artist_track_clean_key', 'album_track_clean_key'],
      dtype='object')

In [206]:
test3 = pd.merge(genius,spotify, on='artist_album_track_key', how='inner')

In [207]:
test3.shape

(20211, 42)

In [208]:
test.drop_duplicates('track_id').shape

(20211, 42)

## Distance Exploration -- Artist Name

#### Baseline Match Rate Before Any Processing

In [209]:
spotify_artist_list_deduped = list(spotify['arist_y'].unique())

In [210]:
#Artists that are in spotify and not in Genius
name_checks_no_genius = genius[~genius['artist'].isin(spotify_artist_list_deduped)].drop_duplicates('artist')['artist']

In [211]:
#Artists that are in spotify and not in Genius
not_matching = genius[~genius['artist'].isin(spotify_artist_list_deduped)].drop_duplicates('artist')['artist'].shape[0]
total = genius['artist'].drop_duplicates().shape[0]
print(f"Matched: {total-not_matching}")
print(f"Match Rate: {1- not_matching/total}")

Matched: 978
Match Rate: 0.7836538461538461


#### Match Rate After Initial Processing

In [112]:
# pd.set_option('display.max_rows', 300)
# pd.set_option('display.max_columns', 300)
# pd.set_option('display.width', 1000)

In [212]:
genius['artist_clean'] = genius['artist'].str.replace('$', 'S')
genius['artist_clean'] = genius['artist_clean'].str.replace("’", "")
genius['artist_clean'] = genius['artist_clean'].str.replace('&', 'and')
genius['artist_clean'] = genius['artist_clean'].str.replace('!','') 
genius['artist_clean'] = genius['artist_clean'].str.replace('([ ]{2,})', ' ', regex = True) 
genius['artist_clean'] = genius['artist_clean'].str.strip()
genius['artist_clean'] = genius['artist_clean'].str.lower()

In [213]:
spotify['artist_clean'] = spotify['arist_y'].str.replace('$', 'S')
spotify['artist_clean'] = spotify['artist_clean'].str.replace("’", "")
spotify['artist_clean'] = spotify['artist_clean'].str.replace('&', 'and')
spotify['artist_clean'] = spotify['artist_clean'].str.replace('!', '') 
spotify['artist_clean'] = spotify['artist_clean'].str.replace('([ ]{2,})', ' ', regex = True) 
spotify['artist_clean'] = spotify['artist_clean'].str.strip()
spotify['artist_clean'] = spotify['artist_clean'].str.lower()

In [214]:
spotify_artist_list_deduped = list(spotify['artist_clean'].unique())

In [215]:
#Artists that are in spotify and not in Genius
name_checks_no_genius = genius[~genius['artist_clean'].isin(spotify_artist_list_deduped)].drop_duplicates('artist_clean')['artist_clean']

In [216]:
#Artists that are in spotify and not in Genius
not_matching = genius[~genius['artist_clean'].isin(spotify_artist_list_deduped)].drop_duplicates('artist_clean')['artist_clean'].shape[0]
total = genius['artist_clean'].drop_duplicates().shape[0]
print(f"Matched: {total-not_matching}")
print(f"Match Rate: {1- not_matching/total}")

Matched: 982
Match Rate: 0.7868589743589743


#### Exploring Fuzzy

In [123]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [124]:
spotify_artist_list_deduped_no_nan = spotify_artist_list_deduped[1:]

In [125]:
name_choices = spotify_artist_list_deduped_no_nan

In [126]:
for name in name_checks_no_genius:
    if(process.extract(name, name_choices, scorer=fuzz.ratio, limit=1)[0][1] > 75):
        print(f"{name}  : {process.extract(name, name_choices, scorer=fuzz.ratio, limit=1)}")

TypeError: expected string or bytes-like object

In [63]:
genius.loc[genius['artist_clean'] == 'grandmaster melle mel and the furious five', 'artist_clean'] = 'grandmaster flash and the furious five'

In [64]:
#Artists that are in spotify and not in Genius
not_matching = genius[~genius['artist_clean'].isin(spotify_artist_list_deduped)].drop_duplicates('artist_clean')['artist_clean'].shape[0]
total = genius['artist_clean'].drop_duplicates().shape[0]
print(f"Matched: {total-not_matching}")
print(f"Match Rate: {1- not_matching/total}")

Matched: 982
Match Rate: 0.7874899759422614


## Distance Exploration -- Album Name

#### Building our Baseline Match Rate

In [217]:
spotify_album_list_deduped = list(spotify['album_name'].unique())

In [218]:
#Album that are in spotify and not in Genius
album_checks_no_genius = genius[~genius['album'].isin(spotify_album_list_deduped)].drop_duplicates('album').dropna()['album']

In [219]:
#Artists that are in spotify and not in Genius
not_matching = genius[~genius['album'].isin(spotify_album_list_deduped)].drop_duplicates('album').dropna()['album'].shape[0]
total = genius['album'].drop_duplicates().dropna().shape[0]
print(f"Matched: {total-not_matching}")
print(f"Match Rate: {1- not_matching/total}")

Matched: 4169
Match Rate: 0.5224310776942356


#### Text Processing for Album Name

In [220]:
string = 'a long red hot los angeles summer night [instrumentals]'

In [221]:
spotify['album_name_clean'] = spotify['album_name'].str.replace('$', 'S')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace(' LP', '')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace(' EP', '')
spotify['album_name_clean'] = spotify['album_name_clean'].str.lower()


spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('edition', '')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('version', '')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('deluxe', '')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('reissue', '')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('remastered', '')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace(' ots', ' ')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('(original motion picture soundtrack)', '')


spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('volume one', 'volume 1')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('volume two', 'volume 2')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('volume three', 'volume 3')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('volume four', 'volume 4')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('volume five', 'volume 5')

spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('volume 1', 'vol 1')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('volume 2', 'vol 2')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('volume 3', 'vol 3')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('volume 4', 'vol 4')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('volume 5', 'vol 5')

spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('vol 1', 'vol1')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('vol 2', 'vol2')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('vol 3', 'vol3')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('vol 4', 'vol4')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('vol 5', 'vol5')

spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('vol1', 'v1')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('vol2', 'v2')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('vol3', 'v3')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('vol4', 'v4')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('vol5', 'v5')



spotify['album_name_clean'] = spotify['album_name_clean'].str.replace("’", "")
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace("'", "")
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('&', 'and')
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('!', '') 
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace("([\(\[]).*?([\)\]])", '', regex = True)
spotify['album_name_clean'] = spotify['album_name_clean'].str.replace('([ ]{2,})', ' ', regex = True) 
spotify['album_name_clean'] = spotify['album_name_clean'].str.strip()

In [222]:
genius['album_name_clean'] = genius['album'].str.replace('$', 'S')

genius['album_name_clean'] = genius['album_name_clean'].str.replace(' EP', '')
genius['album_name_clean'] = genius['album_name_clean'].str.replace(' LP', '')
genius['album_name_clean'] = genius['album_name_clean'].str.lower()



genius['album_name_clean'] = genius['album_name_clean'].str.replace('edition', '')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('version', '')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('deluxe', '')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('reissue', '')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('remastered', '')
genius['album_name_clean'] = genius['album_name_clean'].str.replace(' ots', '')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('(original motion picture soundtrack)', '')

genius['album_name_clean'] = genius['album_name_clean'].str.replace('volume one', 'volume 1')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('volume two', 'volume 2')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('volume three', 'volume 3')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('volume four', 'volume 4')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('volume five', 'volume 5')

genius['album_name_clean'] = genius['album_name_clean'].str.replace('volume 1', 'vol 1')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('volume 2', 'vol 2')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('volume 3', 'vol 3')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('volume 4', 'vol 4')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('volume 5', 'vol 5')

genius['album_name_clean'] = genius['album_name_clean'].str.replace('vol 1', 'vol1')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('vol 2', 'vol2')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('vol 3', 'vol3')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('vol 4', 'vol4')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('vol 5', 'vol5')

genius['album_name_clean'] = genius['album_name_clean'].str.replace('vol1', 'v1')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('vol2', 'v2')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('vol3', 'v3')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('vol4', 'v4')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('vol5', 'v5')


genius['album_name_clean'] = genius['album_name_clean'].str.replace("’", "")
genius['album_name_clean'] = genius['album_name_clean'].str.replace("'", "")
genius['album_name_clean'] = genius['album_name_clean'].str.replace('&', 'and')
genius['album_name_clean'] = genius['album_name_clean'].str.replace('!', '') 
genius['album_name_clean'] = genius['album_name_clean'].str.replace("([\(\[]).*?([\)\]])", '', regex = True)
genius['album_name_clean'] = genius['album_name_clean'].str.replace('([ ]{2,})', ' ', regex = True) 
genius['album_name_clean'] = genius['album_name_clean'].str.strip()
genius['album_name_clean'] = genius['album_name_clean'].str.lower()

In [223]:
spotify_album_list_deduped = list(spotify['album_name_clean'].unique())

In [224]:
#Album that are in spotify and not in Genius
album_checks_no_genius = genius[~genius['album_name_clean'].isin(spotify_album_list_deduped)].drop_duplicates('album_name_clean').dropna()['album_name_clean']

In [225]:
#Artists that are in spotify and not in Genius
not_matching = genius[~genius['album_name_clean'].isin(spotify_album_list_deduped)].drop_duplicates('album_name_clean').dropna()['album_name_clean'].shape[0]
total = genius['album_name_clean'].drop_duplicates().dropna().shape[0]
print(f"Matched: {total-not_matching}")
print(f"Match Rate: {1- not_matching/total}")

Matched: 4865
Match Rate: 0.6208524757529352


##### Distance on Artist_Album

In [226]:
spotify['artist_album_clean'] = spotify['artist_clean'] + "|" + spotify['album_name_clean']  

In [227]:
genius['artist_album_clean'] = genius['artist_clean'] + "|" + genius['album_name_clean']

In [228]:
spotify_artist_album_list_deduped = list(spotify['artist_album_clean'].unique())

In [229]:
##### Album that are in spotify and not in Genius
artist_album_checks_no_genius = genius[~genius['artist_album_clean'].isin(spotify_artist_album_list_deduped)].drop_duplicates('artist_album_clean').dropna()['artist_album_clean']

##### Clean the Track

In [231]:
spotify.columns

Index(['acousticness', 'album_id', 'album_name', 'analysis_url', 'arist_x',
       'artist_id', 'danceability', 'duration_ms', 'energy', 'follower_x',
       'genres_x', 'id', 'image_x', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'pop_x', 'preview_url', 'speechiness', 'tempo',
       'time_signature', 'track_href', 'track_id', 'track_name', 'type', 'uri',
       'valence', 'arist_y', 'genres_y', 'image_y', 'pop_y', 'follower_y',
       'artist_album_track_key', 'artist_clean', 'album_name_clean',
       'artist_album_clean'],
      dtype='object')

In [237]:
genius['song_name_clean'] = genius['song'].str.replace('$', 'S')

In [238]:
spotify['track_name_clean'] = spotify['track_name'].str.replace('$', 'S')

##### Build keys

In [239]:
genius['artist_album_track_clean_key'] = genius['artist_clean'] + '|' + genius['album_name_clean'] + '|' + genius['song_name_clean']
spotify['artist_album_track_clean_key'] =  spotify['artist_clean'] + '|' + spotify['album_name_clean'] + '|' + spotify['track_name_clean']

genius['artist_album_track_clean_key'] = genius['artist_album_track_clean_key'].str.strip()
spotify['artist_album_track_clean_key'] =  spotify['artist_album_track_clean_key'].str.strip()

In [240]:
# Artist_Track field
genius['artist_track_clean_key'] = genius['artist_clean'] + '|'  + genius['song_name_clean']
spotify['artist_track_clean_key'] =  spotify['artist_clean'] + '|' + spotify['track_name_clean']


In [241]:
#Album_Track field
genius['album_track_clean_key'] = genius['album_name_clean'] + '|'  + genius['song_name_clean']
spotify['album_track_clean_key'] =  spotify['album_name_clean'] + '|' + spotify['track_name_clean']


In [176]:
genius.shape, spotify.shape

((45573, 14), (240556, 41))

In [242]:
# merge on Artist_album_track
pd.set_option('display.max_columns', 60)
test1 = pd.merge(genius, spotify, how='inner', on='artist_album_track_key')

In [252]:
test1.shape

(20211, 56)

In [253]:
# merge on Artist_field
test2= pd.merge(genius, spotify, how='inner', on='artist_track_clean_key')

In [254]:
test2.shape

(20290, 56)

In [255]:
test3= pd.merge(genius, spotify, how='inner', on='album_track_clean_key')

In [256]:
test3.shape

(15844, 56)

In [268]:
combined_deduped_df = test2.append(test3).drop_duplicates('track_id')

In [270]:
combined_deduped_df.isnull().sum()

acousticness                          0
album                               369
album_id                              0
album_name                            0
album_name_clean_x                  369
album_name_clean_y                    0
album_track_clean_key             20290
album_track_clean_key_x             686
album_track_clean_key_y             317
analysis_url                          0
arist_x                             917
arist_y                               0
artist                                0
artist_album_clean_x                369
artist_album_clean_y                  0
artist_album_track_clean_key_x      369
artist_album_track_clean_key_y        0
artist_album_track_key_x              0
artist_album_track_key_y              0
artist_clean_x                        0
artist_clean_y                        0
artist_id                             0
artist_track_clean_key              317
artist_track_clean_key_x          20290
artist_track_clean_key_y          20290


In [269]:
combined_deduped_df.drop(columns=[
'album_name_clean_x',
'album_track_clean_key',
'album_track_clean_key_x',
'arist_x',
'arist_y',
'artist_album_clean_x',
'artist_album_track_clean_key_x',
'image_x',
'image_y',   
'pop_x',
'genres_x'
    
    
])

Index(['acousticness', 'album', 'album_id', 'album_name', 'album_name_clean_x',
       'album_name_clean_y', 'album_track_clean_key',
       'album_track_clean_key_x', 'album_track_clean_key_y', 'analysis_url',
       'arist_x', 'arist_y', 'artist', 'artist_album_clean_x',
       'artist_album_clean_y', 'artist_album_track_clean_key_x',
       'artist_album_track_clean_key_y', 'artist_album_track_key_x',
       'artist_album_track_key_y', 'artist_clean_x', 'artist_clean_y',
       'artist_id', 'artist_track_clean_key', 'artist_track_clean_key_x',
       'artist_track_clean_key_y', 'danceability', 'date', 'duration_ms',
       'energy', 'features', 'follower_x', 'follower_y', 'genres_x',
       'genres_y', 'id', 'image_x', 'image_y', 'instrumentalness', 'key',
       'liveness', 'loudness', 'lyrics', 'mode', 'pop_x', 'pop_y',
       'preview_url', 'producers', 'song', 'song_name_clean', 'speechiness',
       'tempo', 'time_signature', 'track_href', 'track_id', 'track_name',
       'trac

In [ ]:
combined_deduped_df.to_csv('stringmatch/temporary_readout.csv', index=False)

# Fuzzy Matching

##### Can we get smarter on our match using Log Reg?


#### Start with Lev Distance to build training set


In [85]:
artist_album_choices = spotify_artist_album_list_deduped[1:]

training_album_genius_name_set = []
training_album_spotify_name_set = []
training_album_lev_score = []

for artist_album in artist_album_checks_no_genius:
    if(process.extract(artist_album, artist_album_choices, scorer=fuzz.ratio, limit=1)[0][1] > 60):
        
        score = process.extract(artist_album, artist_album_choices, scorer=fuzz.ratio, limit=1)[0][1]
        match = process.extract(artist_album, artist_album_choices, scorer=fuzz.ratio, limit=1)[0][0]
        
        training_album_lev_score.append(score)
        
        training_album_genius_name_set.append(artist_album)
        
        training_album_spotify_name_set.append(match)
        
        print(f"{artist_album}  : {match}")

og maco|og maco  : og maco|og zay
og maco|live life  : og maco|live life 2
omarion|cp4  : omarion|o
one be lo|s.o.n.o.g.r.a.m.  : one be lo|sonogram
open mike eagle|rappers will die of natural causes  : open mike eagle|rappers will die of natural causes ()
pharrell williams|hidden figures: the album  : pharrell williams|hidden figures - original score
logic|under pressure ( )  : logic|under pressure
nicki minaj|queen (bonus )  : nicki minaj|queen
nicki minaj|pink friday: roman reloaded  : nicki minaj|pink friday ... roman reloaded
nicki minaj|pink friday: roman reloaded - the re-up  : nicki minaj|pink friday: roman reloaded the re-up
papoose|hoodie season 3  : papoose|papoose season
papoose|mixtape “hoodie season”  : papoose|papoose season
2pac|greatest hits  : skepta|greatest hits
2pac|the rose that grew from concrete vol. 1  : 2pac|the rose that grew from concrete
2pac|r u still down? (remember me)  : 2pac|r u still down? [remember me]
2pac|the rose that grew from concrete (book)  : 

terrace martin|the sex  : terrace martin|times
terrace martin|locke high  : terrace martin|locke high 2
teyana taylor|yandhi  : teyana taylor|vii
the-dream|love hate  : the-dream|lovehate
the-dream|terius nash: 1977  : the-dream|terius nash : 1977
the-dream|love vs. money  : the-dream|love vs money
the-dream|terius nash: 1977 (2012 )  : the-dream|terius nash : 1977
theophilus london|bebey  : theophilus london|vibes
theophilus london|ny theo  : theophilus london|vibes
the weeknd|thursday  : the weeknd|starboy
justin timberlake|the 20/20 experience: the complete experience  : justin timberlake|the 20/20 experience - 2 of 2 ()
justin timberlake|futuresex / lovesounds  : justin timberlake|futuresex/lovesounds
justin timberlake|the 20/20 experience  : justin timberlake|the 20/20 experience ( )
justin timberlake|futuresex / lovesounds ( )  : justin timberlake|futuresex/lovesounds
tragedy khadafi|thug matrix ii  : tragedy khadafi|thug matrix
tragedy khadafi|the war report  : tragedy khadafi|s

bizzy bone|the collection: v2  : bizzy bone|bizzy bone: the collection
bizzy bone|alpha and omega  : bizzy bone|alpha and omega (explicit )
black thought|streams of thought vol. 1  : black thought|streams of thought vol.2
black thought|streams of thought vol. 2  : black thought|streams of thought vol.2
blaze ya dead homie|1 less g n da hood  : blaze ya dead homie|1 less g in the hood
blaze ya dead homie|colton grundy: tha undying  : blaze ya dead homie|colton grundy
blaze ya dead homie|graveyard greats  : blaze ya dead homie|triple threat
blocboy jb|who am i  : blocboy jb|who am i 3
bobby brackins|live good .5  : bobby brackins|to live for
bohemia|skull and bones v2  : bohemia|skull andamp; bones
bohemia|skull and bones  : bohemia|skull andamp; bones
bones|carcass (скелет)  : bones|carcass
booba|autopsie, v4  : booba|autopsie 0
booba|d.u.c  : booba|d.u.c.
bubba sparxxx|dark days, bright nights (newtown )  : bubba sparxxx|dark days, bright nights
c-murder|the truest shit i ever said  : 

chubb rock|chubb rock featuring hitman howie tee  : chubb rock|chubb rock (feat. hitman howie tee)
common|the dreamer/the believer  : common|the dreamer, the believer
cormega|the true meaning  : cormega|true meaning
currensy|pilot talk ii  : currensy|pilot talk: trilogy
currensy|pilot talk  : currensy|pilot talk: trilogy
currensy|pilot talk iii  : currensy|pilot talk: trilogy
dj drama|gangsta grillz: the album  : dj drama|gangsta grillz the album
dj drama|gangsta grillz: the album, v2  : dj drama|gangsta grillz: the album vol. 2
dj premier|beats that collected dust, v2  : dj premier|beats that collected dust vol. 2
dj premier|beats that collected dust  : dj premier|beats that collected dust vol. 1
dj quik|balance and options  : dj quik|balances and options
dj quik|the red tape  : dj quik|the trauma mixtape
dj screw|june 27th  : dj screw|june 27 vol. 2
dj screw|3 n the mornin: part two [blue]  : dj screw|3 n the mornin part two
dj shadow|the mountain has fallen [ep]  : dj shadow|the mou

jennifer lopez|home ()  : jennifer lopez|love?
jeremih|late nights with jeremih  : jeremih|late nights: europe
jermaine dupri|instructions  : jermaine dupri|instructions (clean )
jim jones|harlem: diary of a summer  : jim jones|harlem - diary of a summer
jimmy wopo|muney lane muzik 2  : jimmy wopo|018 mayhemlane muzik
jimmy wopo|muney lane muzik  : jimmy wopo|018 mayhemlane muzik
jme|integrity​  : jme|integrity>
joell ortiz x llmind|human.  : joell ortiz|human
joji|in tongues - ()  : joji|in tongues ()
joji|lost ballads -  : joji|ballads 1
jon connor|season 2  : jon connor|seren
juicy j|rubba band business 2  : juicy j|rubba band business
kohh|梔子 (kuchinashi)  : kohh|kuchinashi
kohh|dirt ii  : kohh|dirt
kyle|beautiful loser  : kyle|beautiful loser ()
kardinal offishall|eye and i  : kardinal offishall|fire and glory
kash doll|brat mail  : kash doll|brat mail (mixtape)
katie got bandz|drillary clinton 2  : katie got bandz|drillary clinton
keith murray|insomnia - the erick sermon compilat

nav|bad habits  : nav|bad habits ()
nate dogg|eminem presents: the re-up  : eminem|eminem presents the re-up
nate dogg|g-funk classics, volumes 1 and 2  : nate dogg|g funk classics, vols. 1 and 2
nate dogg|nate dogg  : nate dogg|nate dogg and friends
nate dogg|g-funk classics vol. 1: ghetto preacher  : nate dogg|g funk classics, vols. 1 and 2
nipsey hussle|the marathon continues  : nipsey hussle|the marathon
nipsey hussle|bullets aint got no names vol. 1  : nipsey hussle|bullets aint got no name vol. 1
obie trice|eminem presents: the re-up  : eminem|eminem presents the re-up
olly murs|24 hrs  : olly murs|24 hrs ()
olly murs|you know i know  : olly murs|you know i know ()
phat kat|carte blanche  : phat kat|carte blanche ( )
philthy rich|nernl 4  : philthy rich|n.e.r.n.l. 4
philthy rich|nernl  : philthy rich|n.e.r.n.l
pitbull|aquaman ()  : pitbull|armando
pokémon|pokémon christmas bash  : pokémon|christmas bash
prince|sign o the times  : prince|sign "o" the times
prince|prince 4ever  : p

In [88]:
artist_album_processing_df = pd.DataFrame({'genius_artist_album':training_album_genius_name_set, 'spotify_artist_album':training_album_spotify_name_set, 'lev': training_album_lev_score})

In [874]:
# artist_album_processing_df.to_csv('stringmatch/trainingdata.csv', index=False)

In [952]:
artist_album_processing_df = pd.read_csv('stringmatch/trainingdatapipes.csv')

#### Construct Training set, include new features


In [953]:
artist_album_processing_df['genius_artist_album'] = artist_album_processing_df['genius_artist_album'].str.replace('|', ' ')
artist_album_processing_df['spotify_artist_album'] = artist_album_processing_df['spotify_artist_album'].str.replace('|', ' ')

In [954]:
artist_album_processing_df['str_len_genius'] = artist_album_processing_df['genius_artist_album'].str.len()

In [955]:
artist_album_processing_df['str_len_difference'] = np.abs(artist_album_processing_df['genius_artist_album'].str.len() - artist_album_processing_df['spotify_artist_album'].str.len())

In [956]:
from scipy.spatial.distance import cosine

In [957]:
def hamdist(str1, str2):
    diffs = 0
    for ch1, ch2 in zip(str1, str2):
        if ch1 != ch2:
            diffs += 1
    return diffs



""" more orthodox and robust implementation """
def dice_coefficient(a, b):
    """dice coefficient 2nt/(na + nb)."""
    if not len(a) or not len(b): return 0.0
    if len(a) == 1:  a=a+u'.'
    if len(b) == 1:  b=b+u'.'
    
    a_bigram_list=[]
    for i in range(len(a)-1):
      a_bigram_list.append(a[i:i+2])
    b_bigram_list=[]
    for i in range(len(b)-1):
      b_bigram_list.append(b[i:i+2])
      
    a_bigrams = set(a_bigram_list)
    b_bigrams = set(b_bigram_list)
    overlap = len(a_bigrams & b_bigrams)
    dice_coeff = overlap * 2.0/(len(a_bigrams) + len(b_bigrams))
    return dice_coeff



In [958]:
hamming_list=[]
dice_list = []
jarowinkler_list = []
for index, row in artist_album_processing_df.iterrows():
    hamming_list.append(hamdist(row['genius_artist_album'], row['spotify_artist_album']))
    dice_list.append(dice_coefficient(row['genius_artist_album'], row['spotify_artist_album']))
    jarowinkler_list.append(distance.get_jaro_distance(row['genius_artist_album'], row['spotify_artist_album'], winkler=True, scaling=0.1))
    

In [959]:
artist_album_processing_df['ham'] = hamming_list
artist_album_processing_df['dice'] = dice_list
artist_album_processing_df['jaro_wink'] = jarowinkler_list

In [960]:
artist_album_processing_df.columns

Index(['genius_artist_album', 'spotify_artist_album', 'lev', 'match',
       'str_len_genius', 'str_len_difference', 'ham', 'dice', 'jaro_wink'],
      dtype='object')

#### Vectorize

In [961]:
from sklearn.feature_extraction.text import TfidfVectorizer#### Construct Training set, include new features


In [962]:
artist_album_processing_df[artist_album_processing_df['match'] =='-'] = '-'

C:\Users\Owen\AppData\Roaming\Python\Python37\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [963]:
artist_album_processing_df = artist_album_processing_df.dropna()

In [964]:
vect = TfidfVectorizer(max_df=0.03, analyzer='word', strip_accents='ascii', sublinear_tf=True)

tdifresults = vect.fit_transform(artist_album_processing_df['genius_artist_album'])

In [965]:
tidf_numeric_df = pd.concat([artist_album_processing_df.drop(columns=['genius_artist_album', 'spotify_artist_album']),
           pd.DataFrame(list(tdifresults.toarray()), columns=vect.get_feature_names())], axis=1)

In [966]:
tidf_numeric_df.shape

(554, 1317)

#### Fit and Train

In [967]:
X = tidf_numeric_df.drop(columns=['match'])
y = tidf_numeric_df['match']

In [968]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=45)

In [969]:
lr = LogisticRegression(penalty='l2', C = .1)

mnb = MultinomialNB()

rf = RandomForestClassifier()

ad = AdaBoostClassifier()

gb = GradientBoostingClassifier()

In [970]:
tidf_numeric_df['match'].value_counts(normalize=True)

1    0.539711
0    0.460289
Name: match, dtype: float64

In [971]:
with warnings.catch_warnings():
   
    # ignore all caught warnings
    warnings.filterwarnings("ignore")
    
    display(cross_val_score(lr, X_train, y_train, cv = 2).mean())
    display(cross_val_score(mnb, X_train, y_train, cv = 2).mean())
    display(cross_val_score(rf, X_train, y_train, cv = 2).mean())
    display(cross_val_score(ad, X_train, y_train, cv = 2).mean())
    display(cross_val_score(gb, X_train, y_train, cv = 2).mean())

0.599500560867475

0.5245446290262272

0.7441643074622082

0.723519042786176

0.7932669195021633

In [972]:
rf.fit(X_train, y_train);
ad.fit(X_train, y_train);
gb.fit(X_train, y_train);

C:\Users\Owen\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [973]:
display(cross_val_score(lr, X_test, y_test, cv = 2).mean())
display(cross_val_score(rf, X_test, y_test, cv = 2).mean())
display(cross_val_score(ad, X_test, y_test, cv = 2).mean())
display(cross_val_score(gb, X_test, y_test, cv = 2).mean())

C:\Users\Owen\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Owen\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6046328169822146

0.7484222604704532

0.7303499713138267

0.7242541594951233

In [941]:
proba_results = pd.DataFrame(gb.predict_proba(X))
proba_results['feature'] = pd.Series(X.columns)

In [942]:
proba_results[0]

0      0.905817
1      0.840098
2      0.905817
3      0.295167
4      0.027232
         ...   
549    0.166613
550    0.718861
551    0.846100
552    0.710707
553    0.833649
Name: 0, Length: 554, dtype: float64

#### Assess

In [943]:
artist_album_processing_df['probs'] = proba_results[1]

In [944]:
proba_results.sort_values(by = 1, ascending=False).head(10)

,0,1,feature
380,0.019350,0.980650,dusty
101,0.023950,0.976050,banks
102,0.023950,0.976050,banner
4,0.027232,0.972768,dice
104,0.029185,0.970815,baptized
452,0.030075,0.969925,flockaveli
515,0.032791,0.967209,goes
116,0.035516,0.964484,beautiful
416,0.036323,0.963677,exotigaz
417,0.037294,0.962706,experience


In [945]:
artist_album_processing_df.sort_values('probs', ascending=False).head(5)

,genius_artist_album,spotify_artist_album,lev,match,str_len_genius,str_len_difference,ham,dice,jaro_wink,probs
380,dj drama gangsta grillz: the album,dj drama gangsta grillz the album,99,1,34,1,10,0.95082,0.99,0.980650
101,spose the audacity,spose the audacity ( ),100,1,18,4,0,0.888889,0.96,0.976050
102,spose happy medium,spose happy medium ( ),100,1,18,4,0,0.894737,0.96,0.976050
4,open mike eagle rappers will die of natural ca...,open mike eagle rappers will die of natural ca...,100,1,50,3,0,0.977778,0.99,0.972768
104,spose preposterously dank,spose preposterously dank (),100,1,25,3,0,0.93617,0.98,0.970815


In [ ]:
# Model Combos:
# Lev
# Ham
# JaroWink
# Dice
# All Dist
# Dist + Domain
# Tfidf
# Everything

#For each combination of features:
    #Fit Gradient Booster
        #Score (Accuracy, AUC)